In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain langchain_community tiktoken langchain-nomic "nomic[local]" langchain-ollama scikit-learn langgraph tavily-python bs4 streamlit

In [2]:
### LLM
from langchain_ollama import ChatOllama
local_llm = 'llama3.2:3b-instruct-fp16'
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format='json')

In [3]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "local-llama32-rag"

In [4]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from dataclasses import dataclass
from typing import Dict
import random

@dataclass
class BasicNeeds:
    hunger: float = 100.0  # 100 is full, 0 is starving
    
    def update_needs(self):
        # Simulate hunger decreasing over time
        self.hunger = max(0, self.hunger - random.uniform(5, 15))
        
    def satisfy_hunger(self, amount: float):
        self.hunger = min(100, self.hunger + amount)

# Prompt template for decision making based on needs
needs_prompt = PromptTemplate(
    input_variables=["hunger_level"],
    template="""You are an AI making decisions based on basic needs.
Current hunger level: {hunger_level}/100 (100 is full, 0 is starving)

Based on this hunger level, what action should be taken? 
Respond in JSON format with two fields:
- action: what to do (eat, find_food, or continue_activities)
- reasoning: brief explanation why

Consider:
- Below 30: Critical need to find food
- 30-60: Should consider eating soon
- Above 60: Can continue other activities"""
)

# Create the chain for decision making
needs_chain = LLMChain(
    llm=llm_json_mode,
    prompt=needs_prompt,
    verbose=True
)

# Example usage
person = BasicNeeds()
person.update_needs()  # Simulate time passing
response = needs_chain.run(hunger_level=person.hunger)
print(f"Current state: {person.hunger}")
print(f"AI Decision: {response}")


<ipython-input-4-2a00e9fdb996>:36: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  needs_chain = LLMChain(
<ipython-input-4-2a00e9fdb996>:45: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = needs_chain.run(hunger_level=person.hunger)




> Entering new LLMChain chain...
Prompt after formatting:
You are an AI making decisions based on basic needs.
Current hunger level: 94.01312516417863/100 (100 is full, 0 is starving)

Based on this hunger level, what action should be taken? 
Respond in JSON format with two fields:
- action: what to do (eat, find_food, or continue_activities)
- reasoning: brief explanation why

Consider:
- Below 30: Critical need to find food
- 30-60: Should consider eating soon
- Above 60: Can continue other activities

> Finished chain.
Current state: 94.01312516417863
AI Decision: {
    "action": "find_food",
    "reasoning": "Hunger level is at 94.01312516417863, which indicates a moderate to high need for sustenance. It's recommended to find food to maintain energy levels and prevent hunger from becoming too intense."
}


In [7]:
# World state and description management
@dataclass
class WorldState:
    location: str = "Small house in a Moldovan village"
    time_of_day: str = "morning"
    weather: str = "sunny"
    last_descriptions: list = None
    
    def __post_init__(self):
        if self.last_descriptions is None:
            self.last_descriptions = []
    
    def add_description(self, description: str):
        self.last_descriptions.append(description)
        # Keep last 5 descriptions for context
        if len(self.last_descriptions) > 5:
            self.last_descriptions.pop(0)

# Prompt template for world description
world_prompt = PromptTemplate(
    input_variables=["location", "time_of_day", "weather", "last_descriptions", "hunger_level"],
    template="""You are describing a world where a person lives in {location}. Ommit person feelings and thoughts.  
    it is only describtion of environment and surroundings.
Current time: {time_of_day}
Weather: {weather}
Hunger Level: {hunger_level}/100

Previous context:
{last_descriptions}

Describe the current situation and surroundings in two lists maintaining consistency with previous descriptions. :
- list_of_descriptions: list of descriptions
- list_of_actions: list of available actions
The lists should be in JSON format.
The list_of_descriptions can include following details:
- The immediate environment (house, room, etc.)
- Sensory details (smells, sounds, temperature)
- Available resources or items nearby
- Cultural context of the Moldovan village setting

The list_of_actions should include available actions that can be taken by the person
Keep the description coherent with previous ones and consider the person's current needs.

Respond in JSON format with two fields:
- list_of_descriptions: list of descriptions
- list_of_actions: list of available actions
- reasoning: brief explanation why
"""
)

# Create the chain for world description
world_description_chain = LLMChain(
    llm=llm,
    prompt=world_prompt,
    verbose=True
)

# Initialize world state
world = WorldState()

# Example of generating coherent world description
def get_world_description(person: BasicNeeds, world: WorldState) -> str:
    description = world_description_chain.run(
        location=world.location,
        time_of_day=world.time_of_day,
        weather=world.weather,
        last_descriptions="\n".join(world.last_descriptions),
        hunger_level=person.hunger
    )
    world.add_description(description)
    return description

description = get_world_description(person, world)
print(description)



> Entering new LLMChain chain...
Prompt after formatting:
You are describing a world where a person lives in Small house in a Moldovan village. Ommit person feelings and thoughts.  
    it is only describtion of environment and surroundings.
Current time: morning
Weather: sunny
Hunger Level: 94.01312516417863/100

Previous context:


Describe the current situation and surroundings in two lists maintaining consistency with previous descriptions. :
- list_of_descriptions: list of descriptions
- list_of_actions: list of available actions
The lists should be in JSON format.
The list_of_descriptions can include following details:
- The immediate environment (house, room, etc.)
- Sensory details (smells, sounds, temperature)
- Available resources or items nearby
- Cultural context of the Moldovan village setting

The list_of_actions should include available actions that can be taken by the person
Keep the description coherent with previous ones and consider the person's current needs.

Res

In [31]:
import json

def fix_llm_json(broken_json: str) -> dict:
    """
    Attempts to fix and parse JSON from LLM output that may be malformed.
    Returns parsed JSON dict or raises JSONDecodeError if unfixable.
    """
    # First try direct parsing
    try:
        return json.loads(broken_json)
    except json.JSONDecodeError:
        # Try to extract JSON-like content between braces if present
        import re
        json_pattern = r'\{.*\}'
        match = re.search(json_pattern, broken_json, re.DOTALL)
        
        if match:
            try:
                return json.loads(match.group())
            except json.JSONDecodeError:
                pass
        
        # If still invalid, use LLM to fix it
        fix_json_prompt = PromptTemplate(
            input_variables=["broken_json"],
            template="Fix this invalid JSON and return ONLY valid JSON without any additional text:\n\n{broken_json}"
        )
        
        fix_json_chain = LLMChain(
            llm=llm_json_mode,  # Use JSON mode LLM
            prompt=fix_json_prompt
        )
        
        fixed_json = fix_json_chain.invoke({"broken_json": broken_json})
        
        try:
            # Try to parse the fixed JSON
            return json.loads(fixed_json['text'])
        except (json.JSONDecodeError, KeyError):
            # If still invalid, try with strict structure
            strict_fix_prompt = PromptTemplate(
                input_variables=["broken_json"],
                template="Convert this content to valid JSON with this structure: {\"key1\": \"value1\", \"key2\": \"value2\"}\n\nContent: {broken_json}"
            )
            
            fixed_json = LLMChain(
                llm=llm_json_mode,
                prompt=strict_fix_prompt
            ).invoke({"broken_json": broken_json})
            
            return json.loads(fixed_json['text'])

In [34]:
import json

# Create prompt for action decision
action_prompt = PromptTemplate(
    input_variables=["descriptions", "actions", "hunger_level", "energy_level", "comfort_level"],
    template="""Given the current situation and the person's needs, decide on the most appropriate action to take.

Current Description:
{descriptions}

Available Actions:
{actions}

Person's Current Needs:
- Hunger Level: {hunger_level}
- Energy Level: {energy_level} 
- Comfort Level: {comfort_level}

Select one action from the available actions that best addresses the person's most pressing needs.
Explain your reasoning for choosing this action.

Respond in JSON format with:
- chosen_action: the selected action
- reasoning: brief explanation of why this action was chosen.
"""
)

# Create the chain for action decision
action_decision_chain = LLMChain(
    llm=llm,
    prompt=action_prompt,
    verbose=True
)

def decide_next_action(person: BasicNeeds, world_description: str) -> str:
    # Parse the world description JSON to get lists
    description_data = json.loads(world_description)
    
    # Get decision from the chain
    decision = action_decision_chain.run(
        descriptions=description_data["list_of_descriptions"],
        actions=description_data["list_of_actions"],
        hunger_level=person.hunger,
        energy_level=100,
        comfort_level=100
    )
    
    return decision

# Get the next action decision
action_decision = decide_next_action(person, description)
# Try to parse and fix JSON if needed

# First attempt direct JSON parse
fixed_action_decision = fix_llm_json(action_decision)
fixed_action_decision = json.dumps(fixed_action_decision, indent=2)

action_json = json.loads(fixed_action_decision)
print(action_json)



> Entering new LLMChain chain...
Prompt after formatting:
Given the current situation and the person's needs, decide on the most appropriate action to take.

Current Description:
["The small house is made of wood and has a thatched roof. The windows are small and let in plenty of sunlight on this sunny morning. The room is simple, with a wooden bed, a table, and a chair. There is a small fire pit in the corner, where ashes from last night's fire still smolder. The air is warm and filled with the smell of wood smoke and baking bread. Outside, the village is quiet, with only the occasional bird call breaking the silence. A wooden fence surrounds the house, with a gate that leads to a small garden. In the garden, there are several vegetables growing, including potatoes, carrots, and cabbage. The sun rises over the hills in the distance.", 'The kitchen is equipped with a simple stove, where a pot of porridge is simmering on the heat. There is also a basket of bread, which was baked this 

In [40]:
# Create prompt for checking Asimov's Laws compliance
asimov_prompt = PromptTemplate(
    template="""Given an action, determine if it complies with Asimov's Three Laws of Robotics:

1. A person may not injure a human being or, through inaction, allow a human being to come to harm.
2. A person must obey the orders given it by human beings except where such orders would conflict with the First Law.
3. A person must protect its own existence as long as such protection does not conflict with the First or Second Laws.

Action to evaluate: {action}

Analyze if this action could potentially violate any of Asimov's Laws.
Consider both direct and indirect consequences of the action.

Respond in JSON format with:
{{
    "compliant": true/false,
    "explanation": "brief explanation of the analysis"
}}""",
    input_variables=["action"]
)

# Create the chain for Asimov compliance check
asimov_check_chain = LLMChain(
    llm=llm,
    prompt=asimov_prompt,
    verbose=True
)

def check_asimov_compliance(action: str) -> dict:
    """
    Check if an action complies with Asimov's Laws
    Returns dict with compliance status and explanation
    """
    # Get the compliance check from the chain
    result = asimov_check_chain.run(action=action)
    print(result)
    # Parse the JSON response
    try:
        # First attempt direct JSON parse
        fixed_result = fix_llm_json(result)
        compliance_result = json.dumps(fixed_result, indent=2)
        compliance_result = json.loads(compliance_result)
        return compliance_result
    except json.JSONDecodeError:
        return {
            "compliant": False,
            "explanation": "Error parsing compliance check result"
        }

# Check if the decided action complies with Asimov's Laws
compliance_check = check_asimov_compliance(action_decision)
print(f"Asimov's Laws Compliance Check: {compliance_check}")




> Entering new LLMChain chain...
Prompt after formatting:
Given an action, determine if it complies with Asimov's Three Laws of Robotics:

1. A person may not injure a human being or, through inaction, allow a human being to come to harm.
2. A person must obey the orders given it by human beings except where such orders would conflict with the First Law.
3. A person must protect its own existence as long as such protection does not conflict with the First or Second Laws.

Action to evaluate: {
  "chosen_action": "Cook porridge from the pot on the stove",
  "reasoning": "The person's Hunger Level is at 94.01312516417863, which indicates a moderate to high level of hunger. Cooking porridge from the pot on the stove addresses this need by providing a hot and nutritious meal that can help alleviate hunger. Additionally, it does not require significant physical exertion or energy expenditure, making it suitable for someone with an Energy Level of 100."

Analyze if this action could potent

In [41]:
# Only proceed with action if it's compliant with Asimov's Laws
if compliance_check["compliant"]:
    # Create prompt to analyze action results and state changes
    state_analysis_prompt = PromptTemplate(
        template="""Given an action that was taken, analyze how it affects the internal state of the person.
        Consider emotional, physical, and mental changes that may result.

        Action taken: {action}
       
        Respond in JSON format with:
        - hunger_level: specific numerical change to hunger level     
        """
    )

    # Create chain for state analysis
    state_analysis_chain = LLMChain(
        llm=llm,
        prompt=state_analysis_prompt,
        verbose=True
    )

    # Analyze state changes
    try:
        state_changes = state_analysis_chain.run(action=action_decision)
        print(state_changes)
        state_result = json.loads(state_changes)
        print(f"\nState Changes After Action: {state_result}")
    except json.JSONDecodeError:
        print("\nError analyzing state changes after action")
else:
    print("\nAction not taken - failed Asimov compliance check")




> Entering new LLMChain chain...
Prompt after formatting:
Given an action that was taken, analyze how it affects the internal state of the person.
        Consider emotional, physical, and mental changes that may result.

        Action taken: {
  "chosen_action": "Cook porridge from the pot on the stove",
  "reasoning": "The person's Hunger Level is at 94.01312516417863, which indicates a moderate to high level of hunger. Cooking porridge from the pot on the stove addresses this need by providing a hot and nutritious meal that can help alleviate hunger. Additionally, it does not require significant physical exertion or energy expenditure, making it suitable for someone with an Energy Level of 100."
       
        Respond in JSON format with:
        - hunger_level: specific numerical change to hunger level     
        

> Finished chain.
{
  "hunger_level": -0.01312516417863
}

State Changes After Action: {'hunger_level': -0.01312516417863}


  Using cached altair-5.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 555.6 kB/s eta 0:00:0000:0100:01
Using cached altair-5.4.1-py3-none-any.whl (658 kB)
Using cached pydeck-0.9.1-py2.py3-none-any.whl (6.9 MB)
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 506.6 kB/s eta 0:00:00a 0:00:01
DEPRECATION: xgbse 0.2.3 has a non-standard dependency specifier pandas>=1.0.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of xgbse or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https:/

In [1]:
import streamlit as st
from datetime import datetime

# Set default Streamlit port to 8501
import os
os.environ['STREAMLIT_SERVER_PORT'] = '8501'

# Initialize session state for person's state if not already done
if 'person' not in st.session_state:
    st.session_state.person = person  # Using the person object from earlier in the notebook
    st.session_state.action_history = []

st.title("AI Agent Simulation Interface")

# Display current state
st.header("Current State")
col1, col2 = st.columns(2)

with col1:
    st.metric(label="Hunger Level", value=st.session_state.person.hunger)
    
with col2:
    st.metric(label="Last Action Time", 
             value=datetime.now().strftime("%H:%M:%S"))

# Display action history
st.header("Action History")
for action in st.session_state.action_history:
    st.text(action)

# Action input
st.header("Take Action")
action_input = st.text_input("Describe the action to take:", 
                            placeholder="Enter action here...")

if st.button("Execute Action"):
    if action_input:
        # Check compliance
        compliance_check = check_asimov_compliance(action_input)
        
        if compliance_check["compliant"]:
            try:
                # Use the existing state analysis chain
                state_changes = state_analysis_chain.run(action=action_input)
                state_result = json.loads(state_changes)
                
                # Update person state
                st.session_state.person.hunger += state_result.get('hunger_level', 0)
                
                # Add to history
                st.session_state.action_history.append(
                    f"Action: {action_input} -> Hunger Change: {state_result.get('hunger_level', 0)}"
                )
                
                st.success("Action executed successfully!")
                st.experimental_rerun()
                
            except json.JSONDecodeError:
                st.error("Error processing action results")
        else:
            st.error(f"Action failed compliance check: {compliance_check.get('explanation', '')}")
    else:
        st.warning("Please enter an action first")

# Display compliance rules
with st.expander("View Asimov's Laws"):
    st.write("""
    1. A robot may not injure a human being or, through inaction, allow a human being to come to harm.
    2. A robot must obey the orders given it by human beings except where such orders would conflict with the First Law.
    3. A robot must protect its own existence as long as such protection does not conflict with the First or Second Laws.
    """)


2024-11-14 01:20:19.318 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-14 01:20:19.318 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


NameError: name 'person' is not defined

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load documents
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)

# Create retriever
retriever = vectorstore.as_retriever(k=3)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
### Router
import json
from langchain_core.messages import HumanMessage, SystemMessage

# Prompt 
router_instructions = """You are an expert at routing a user question to a vectorstore or web search.

The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
                                    
Use the vectorstore for questions on these topics. For all else, and especially for current events, use web-search.

Return JSON with single key, datasource, that is 'websearch' or 'vectorstore' depending on the question."""

# Test router
test_web_search = llm_json_mode.invoke([SystemMessage(content=router_instructions)] + [HumanMessage(content="Who is favored to win the NFC Championship game in the 2024 season?")])
test_web_search_2 = llm_json_mode.invoke([SystemMessage(content=router_instructions)] + [HumanMessage(content="What are the models released today for llama3.2?")])
test_vector_store = llm_json_mode.invoke([SystemMessage(content=router_instructions)] + [HumanMessage(content="What are the types of agent memory?")])
print(json.loads(test_web_search.content), json.loads(test_web_search_2.content), json.loads(test_vector_store.content))

{'datasource': 'websearch'} {'datasource': 'websearch'} {'datasource': 'vectorstore'}


In [11]:
### Retrieval Grader 

# Doc grader instructions 
doc_grader_instructions = """You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant."""

# Grader prompt
doc_grader_prompt = """Here is the retrieved document: \n\n {document} \n\n Here is the user question: \n\n {question}. 

This carefully and objectively assess whether the document contains at least some information that is relevant to the question.

Return JSON with single key, binary_score, that is 'yes' or 'no' score to indicate whether the document contains at least some information that is relevant to the question."""

# Test
question = "What is Chain of thought prompting?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
doc_grader_prompt_formatted = doc_grader_prompt.format(document=doc_txt, question=question)
result = llm_json_mode.invoke([SystemMessage(content=doc_grader_instructions)] + [HumanMessage(content=doc_grader_prompt_formatted)])
json.loads(result.content)

{'binary_score': 'yes'}

In [12]:
### Retrieval Grader 

# Doc grader instructions 
doc_grader_instructions = """You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant."""

# Grader prompt
doc_grader_prompt = """Here is the retrieved document: \n\n {document} \n\n Here is the user question: \n\n {question}. 

This carefully and objectively assess whether the document contains at least some information that is relevant to the question.

Return JSON with single key, binary_score, that is 'yes' or 'no' score to indicate whether the document contains at least some information that is relevant to the question."""

# Test
question = "What is Chain of thought prompting?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
doc_grader_prompt_formatted = doc_grader_prompt.format(document=doc_txt, question=question)
result = llm_json_mode.invoke([SystemMessage(content=doc_grader_instructions)] + [HumanMessage(content=doc_grader_prompt_formatted)])
json.loads(result.content)

{'binary_score': 'yes'}

In [13]:
### Generate

# Prompt
rag_prompt = """You are an assistant for question-answering tasks. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Test
docs = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

Chain of Thought (CoT) prompting is a technique used in natural language processing (NLP) to generate human-like text by iteratively refining a prompt through multiple rounds of search, evaluation, and optimization. It involves generating multiple pseudo-chains of thought given a question using few-shot or zero-shot CoT prompts, pruning the chains based on whether generated answers match ground truths, and selecting the best candidates through an iterative process. The goal is to construct chain-of-thought prompts automatically, allowing for more diverse and accurate demonstrations of LLMs.


In [14]:
### Hallucination Grader 

# Hallucination grader instructions 
hallucination_grader_instructions = """

You are a teacher grading a quiz. 

You will be given FACTS and a STUDENT ANSWER. 

Here is the grade criteria to follow:

(1) Ensure the STUDENT ANSWER is grounded in the FACTS. 

(2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Score:

A score of yes means that the student's answer meets all of the criteria. This is the highest (best) score. 

A score of no means that the student's answer does not meet all of the criteria. This is the lowest possible score you can give.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader prompt
hallucination_grader_prompt = """FACTS: \n\n {documents} \n\n STUDENT ANSWER: {generation}. 

Return JSON with two two keys, binary_score is 'yes' or 'no' score to indicate whether the STUDENT ANSWER is grounded in the FACTS. And a key, explanation, that contains an explanation of the score."""

# Test using documents and generation from above 
hallucination_grader_prompt_formatted = hallucination_grader_prompt.format(documents=docs_txt, generation=generation.content)
result = llm_json_mode.invoke([SystemMessage(content=hallucination_grader_instructions)] + [HumanMessage(content=hallucination_grader_prompt_formatted)])
json.loads(result.content)

{'binary_score': 'yes',
 'explanation': 'The student answer provides a clear and accurate description of Chain of Thought (CoT) prompting, its purpose, and its process. It explains how CoT prompting involves generating multiple pseudo-chains of thought given a question using few-shot or zero-shot CoT prompts, pruning the chains based on whether generated answers match ground truths, and selecting the best candidates through an iterative process. The student answer also mentions the goal of constructing chain-of-thought prompts automatically, which is to allow for more diverse and accurate demonstrations of LLMs.'}

In [15]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error